In [1]:
# ! pip install bamt

In [2]:
from dataclasses import dataclass
from enum import Enum, auto
from typing import Self
from uuid import uuid4
from bamt.preprocessors import Preprocessor
import pandas as pd
from sklearn import preprocessing as pp
from bamt.networks import HybridBN


class Sex(Enum):
    male = auto()
    female = auto()


class MaritalStatus(Enum):
    single = auto()
    maried = auto()


class Education(Enum):
    secondary = auto()  # среднее
    secondary_proffesional = auto()
    higher = auto()


@dataclass
class Agent:
    id = uuid4()
    age: int
    sex: Sex
    salary: int
    education: Education | str
    mortgage_dept: int | None = None
    marital_status: MaritalStatus | str | None = None
    children: list[Self] | None = None
    parents: list[Self] | None = None

In [3]:
import pandas as pd


df = pd.read_csv("salary.csv")

In [4]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

In [5]:
def parse_sex(v):
    if v==" Male":
        return Sex.male
    if v==" Female":
        return Sex.female
    return None
population = []
for index, row in df.iterrows():
    agent = Agent(age=row['age'],
                  sex=parse_sex(row["sex"]),
                  education=row["education"],
                  salary=1
                  
                  )
    population.append(agent)

In [6]:
population[23]

Agent(age=43, sex=<Sex.male: 1>, salary=1, education=' 11th', mortgage_dept=None, marital_status=None, children=None, parents=None)

In [7]:
rlms = pd.read_excel("Data_RLMS.xlsx")
rlms.columns

Index(['idind', 'psu', 'status', 'age', 'male', 'industry', 'lnwage', 'public',
       'internet', 'children', 'urban', 'educ', 'id1', 'id2', 'id3', 'id4',
       'id5', 'id6', 'id7', 'id8', 'id9', 'id10', 'id11', 'id12', 'id13',
       'id14', 'id15', 'id16', 'id17', 'id18', 'id19', 'id20', 'id21', 'id22',
       'id23', 'id24', 'id25', 'id26', 'id27', 'id28', 'id29', 'id30', 'id31',
       'id32', 'id33', 'id34', 'id35', 'id36', 'id37', 'id38', 'id39', 'id40',
       'id41', 'id42', 'id43', 'id44', 'id45', 'id46', 'id47', 'id48', 'id49',
       'id50', 'id51', 'id52', 'id53', 'id54', 'id55', 'id56', 'id57', 'id58',
       'id59', 'id60', 'id61', 'id62', 'id63', 'id64', 'id65', 'id66', 'id67',
       'id68', 'id69', 'id70', 'id71', 'id72', 'id73', 'id74', 'id75'],
      dtype='object')

In [8]:
data = rlms[["age", "male", "industry", "lnwage",
             "public", "children", "urban", "educ"]]

data = rlms[["age",  "lnwage",
             ]]

In [9]:
# set encoder and discretizer
encoder = pp.LabelEncoder()
discretizer = pp.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')

# create preprocessor object with encoder and discretizer
p = Preprocessor([('encoder', encoder), ('discretizer', discretizer)])

# discretize data for structure learning
discretized_data, est = p.apply(data)

# get information about data
info = p.info

In [10]:
info

{'types': {'age': 'disc_num', 'lnwage': 'cont'}, 'signs': {'lnwage': 'pos'}}

In [11]:
# initialize network object
bn = HybridBN(use_mixture=True, has_logit=True)

# add nodes to network
bn.add_nodes(info)

# using mutual information as scoring function for structure learning
bn.add_edges(discretized_data, 
            #  scoring_function=('MI',)
             )

# or use evolutionary algorithm to learn structure

bn.add_edges(discretized_data)

bn.fit_parameters(data)

  0%|          | 0/1000000 [00:00<?, ?it/s]

2024-10-27 14:54:37,721 | ERROR    | builders_base.py-get_family-0082 | Edges list is None


  0%|          | 0/1000000 [00:00<?, ?it/s]

2024-10-27 14:54:37,742 | ERROR    | builders_base.py-get_family-0082 | Edges list is None


In [12]:
bn.validate(info)

True

In [13]:
bn.save("pop.json")

True

In [14]:
sampled_data = bn.sample(1000, progress_bar=False)

In [15]:
sampled_data

,age,lnwage
0,54,9.981127
1,26,10.524094
2,38,9.400916
3,32,10.790339
4,53,10.663289
...,...,...
995,44,9.287571
996,38,10.346601
997,35,10.522201
998,32,10.043810
